In [25]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree
from nltk import ngrams
import json

In [26]:
# import internal modules
import file_path_management as fpath
import public_library as plib
import extract_info
import parameters as params
import download_and_process_pdf as dpp

In [27]:
def pdf2text(pdf_path, text_path, start_page, end_page, text_length_to_extract):    
    try:
        # creating a pdf reader object
        reader = PyPDF2.PdfReader(pdf_path)
        
        # printing number of pages in pdf file
        page_max = len(reader.pages)
        print(page_max)
        
        # getting a specific page from the pdf file
        text = ""
        
        for i in range(start_page, page_max):
            page = reader.pages[i]
            text = text + " ".join(page.extract_text().splitlines())
            if len(text.split()) > text_length_to_extract:
                break

        with open(text_path, "w") as f:
            f.write(text)
        f.close()
        return True
    except:
        return False
# --------------------start of test code--------------------
# index = 1
# pdf_folder = fpath.pdf_folder
# text_folder = fpath.text_folder
# start_page = 0
# end_page = 5
# text_length_to_extract = 2000

# pdf_file_name = str(index) + ".pdf"
# pdf_path = os.path.join(pdf_folder, pdf_file_name)
# text_path = os.path.join(text_folder, pdf_file_name.split(".pdf")[0] + ".txt")
# # print(pdf_path)
# # print(text_path)
# pdf2text(pdf_path, text_path, start_page, end_page, text_length_to_extract)
# ---------------------end of test code---------------------

In [28]:
def json2text(json_path, text_path, text_length_to_extract):    
    try:
        with open(json_path, "r") as f:
            json_file = json.load(f)
            text = json_file["originalText"]

        title = json_file["dc:title"]
    
        text = text.split(title)[1]

        words = text.split()
        text = ' '.join(words[:text_length_to_extract])

        with open(text_path, "w") as f:
            f.write(text)
        f.close()

        return True
    except:
        return False
# --------------------start of test code--------------------
# index = 1
# pdf_folder = fpath.pdf_folder
# text_folder = fpath.text_folder
# start_page = 0
# end_page = 5
# text_length_to_extract = 2000

# pdf_file_name = str(index) + ".pdf"
# pdf_path = os.path.join(pdf_folder, pdf_file_name)
# text_path = os.path.join(text_folder, pdf_file_name.split(".pdf")[0] + ".txt")
# # print(pdf_path)
# # print(text_path)
# pdf2text(pdf_path, text_path, start_page, end_page, text_length_to_extract)
# ---------------------end of test code---------------------

In [29]:
def test_pdf(pdf_path):       
    # opens the file for reading
    with open(pdf_path, 'rb') as p:
        txt = (p.readlines())

    actual_line = len(txt)
    
    for i, x in enumerate(txt[::-1]):
        if b'%%EOF' in x:
            actual_line = len(txt)-i
            # print(f'EOF found at line position {-i} = actual {actual_line}, with value {x}')
            break
    
    if actual_line != len(txt):
        # get the new list terminating correctly
        txtx = txt[:actual_line]

        # write to new pdf
        with open(pdf_path, 'wb') as f:
            f.writelines(txtx)
        f.close()

    fixed_pdf = PyPDF2.PdfReader(pdf_path)

    page_max = len(fixed_pdf.pages)

    if page_max < 5:
        print(page_max)
        return False
    else:
        return True
# --------------------start of test code--------------------
# test_pdf("/Volumes/DIDIHOU/pdfs/645.pdf")
# ---------------------end of test code---------------------

In [30]:
# step 3: test the downloaded pdfs
input_path = fpath.poten_litera_db
df = pd.read_csv(input_path, header=None, sep=',')
df.columns = ["INDEX", "DOI", "PMID", "PMCID", "FULL_TEXT_URL", "FULL_TEXT_SOURCE", "PDF_URL", "PDF_SOURCE", "TITLE", "ABSTRACT", "KEYWORDS"]

pdf_folder = fpath.pdf_folder

start = 0
end = 10776

for ind in range(start, end):
    index = str(int(df.at[ind, "INDEX"]))
    pdf_file_name = str(index) + ".pdf"
    json_file_name = str(index) + ".json"
    pdf_path = os.path.join(pdf_folder, pdf_file_name)
    json_path = os.path.join(pdf_folder, json_file_name)

    try:
        if os.path.exists(json_path):
            print(ind, index)
            continue
        elif os.path.exists(pdf_path):
            if test_pdf(pdf_path):
                print(ind, index)
            else:
                print("\n")
                print("PDF LEGNTH < 5")
                print(df.at[ind, "INDEX"], df.at[ind, "DOI"], df.at[ind, "PMID"], df.at[ind, "PMCID"])
                print(df.at[ind, "FULL_TEXT_URL"], df.at[ind, "FULL_TEXT_SOURCE"])
                print(df.at[ind, "PDF_URL"], df.at[ind, "PDF_SOURCE"])
                print(ind, index)
                print("\n")
        else:
            print("\n")
            print("PDF NOT AVAILABLE")
            print(df.at[ind, "INDEX"], df.at[ind, "DOI"], df.at[ind, "PMID"], df.at[ind, "PMCID"])
            print(df.at[ind, "FULL_TEXT_URL"], df.at[ind, "FULL_TEXT_SOURCE"])
            print(df.at[ind, "PDF_URL"], df.at[ind, "PDF_SOURCE"])
            print(ind, index)
            print("\n")
    except:
        print("\n")
        print("PDF Corrupted")
        print(df.at[ind, "INDEX"], df.at[ind, "DOI"], df.at[ind, "PMID"], df.at[ind, "PMCID"])
        print(df.at[ind, "FULL_TEXT_URL"], df.at[ind, "FULL_TEXT_SOURCE"])
        print(df.at[ind, "PDF_URL"], df.at[ind, "PDF_SOURCE"])
        print(ind, index)
        print("\n")

0 0
1 1
2 2
3 4
4


PDF LEGNTH < 5
5 10.1038/372770a0 7695716.0 nan
https://www.nature.com/articles/372770a0 www.nature.com
https://www.nature.com/articles/372770a0.pdf www.nature.com
4 5


5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
21 22
22 23
23 24
24 25
25 26
26 27
27 28


PDF NOT AVAILABLE
29 10.1017/s0952523800004922 1375095.0 nan
https://www.cambridge.org/core/journals/visual-neuroscience/article/abs/segregated-thalamocortical-pathways-to-inferior-parietal-and-inferotemporal-cortex-in-macaque-monkey/D15415AF6BF28483F6B6676F4EA32B81 www.cambridge.org
nan nan
28 29


29 30
30 31
31 32
32 33
33 34
34 35
35 36
36 37
37 38
38 39
39 40
40 41


PDF NOT AVAILABLE
42 10.1016/j.neuroscience.2007.02.033 17395383.0 nan
https://linkinghub.elsevier.com/retrieve/pii/S030645220700187X linkinghub.elsevier.com
://linkinghub.elsevier.com/ linkinghub.elsevier.com
41 42




PDF NOT AVAILABLE
43 10.1017/s0952523800002406 8038123.0 nan
https://www.cambridge.o

2023-09-18 16:32:26 130-177.eduroam.rwth-aachen.de PyPDF2.generic._data_structures[5571] WARNING Multiple definitions in dictionary at byte 0x7da4 for key /MediaBox


238 248
239 249


PDF NOT AVAILABLE
250 nan 8337932.0 nan
https://europepmc.org/article/med/8337932 europepmc.org
://europepmc.org/ europepmc.org
240 250


241 251
242 252
243 253
2


PDF LEGNTH < 5
254 10.3389/fnsys.2015.00039 25852498.0 PMC4362213
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4362213/ www.ncbi.nlm.nih.gov
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4362213/pdf/fnsys-09-00039.pdf www.ncbi.nlm.nih.gov
244 254


245 255
246 256
247 257
248 258


PDF NOT AVAILABLE
259 10.1016/s0079-6123(02)36031-x 12143395.0 nan
https://linkinghub.elsevier.com/retrieve/pii/S007961230236031X linkinghub.elsevier.com
://linkinghub.elsevier.com/ linkinghub.elsevier.com
249 259


250 260


PDF NOT AVAILABLE
261 10.1016/b978-0-08-042274-9.50017-1 nan nan
https://linkinghub.elsevier.com/retrieve/pii/B9780080422749500171 linkinghub.elsevier.com
://linkinghub.elsevier.com/ linkinghub.elsevier.com
251 261


252 262
253 263
254 264
255 265
256 266
257 267


PDF NOT AVAILABLE
268 10.1007/978-1-349-1

2023-09-18 16:32:48 130-177.eduroam.rwth-aachen.de PyPDF2.generic._data_structures[5571] WARNING Multiple definitions in dictionary at byte 0x2f0aa for key /MediaBox
2023-09-18 16:32:48 130-177.eduroam.rwth-aachen.de PyPDF2.generic._data_structures[5571] WARNING Multiple definitions in dictionary at byte 0x2f310 for key /MediaBox
2023-09-18 16:32:48 130-177.eduroam.rwth-aachen.de PyPDF2.generic._data_structures[5571] WARNING Multiple definitions in dictionary at byte 0x2f591 for key /MediaBox
2023-09-18 16:32:48 130-177.eduroam.rwth-aachen.de PyPDF2.generic._data_structures[5571] WARNING Multiple definitions in dictionary at byte 0x2f7fa for key /MediaBox
2023-09-18 16:32:48 130-177.eduroam.rwth-aachen.de PyPDF2.generic._data_structures[5571] WARNING Multiple definitions in dictionary at byte 0x2f990 for key /MediaBox
2023-09-18 16:32:48 130-177.eduroam.rwth-aachen.de PyPDF2.generic._data_structures[5571] WARNING Multiple definitions in dictionary at byte 0x2fb5e for key /MediaBox
2023

923 946
924 947
925 948
926 949
927 950
928 951
929 952
930 953
931 954
932 955
933 956
934 957
935 958
936 959


PDF NOT AVAILABLE
960 10.1016/j.brainres.2006.07.043 16935269.0 nan
https://linkinghub.elsevier.com/retrieve/pii/S0006899306021457 linkinghub.elsevier.com
://linkinghub.elsevier.com/ linkinghub.elsevier.com
937 960


938 961
939 962
940 963
941 964
942 965
943 966
944 967
945 968
946 969
947 970
948 971
949 972
950 973
951 974
952 975
953 976
954 977
955 978
956 979
957 980
958 981
959 983
960 984
961 985
962 986
963 987
964 989
965 990
966 991
967 992


PDF NOT AVAILABLE
993 10.1016/s0197-4580(03)00044-7 14749136.0 nan
https://linkinghub.elsevier.com/retrieve/pii/S0197458003000447 linkinghub.elsevier.com
://linkinghub.elsevier.com/ linkinghub.elsevier.com
968 993


969 994


PDF NOT AVAILABLE
995 10.1016/j.jchemneu.2004.09.003 15589702.0 nan
https://linkinghub.elsevier.com/retrieve/pii/S0891061804001218 linkinghub.elsevier.com
://linkinghub.elsevier.com/ linkinghub.elsevie

...

10073 10277
10074 10278
10075 10279
10076 10280
10077 10281
10078 10282
10079 10283
10080 10284
10081 10285
4


PDF LEGNTH < 5
10286 10.1136/jnnp.49.6.702 3090207.0 PMC1028855
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1028855/ www.ncbi.nlm.nih.gov
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1028855/pdf/jnnpsyc00098-0092.pdf www.ncbi.nlm.nih.gov
10082 10286


10083 10287
10084 10288
10085 10289
10086 10290
10087 10291
10088 10292
10089 10293
10090 10294
10091 10295
10092 10296
10093 10297
10094 10298
10095 10299
10096 10300
10097 10301
10098 10302
10099 10303
10100 10304
10101 10305
10102 10306
10103 10307
10104 10308
10105 10309
10106 10310
10107 10311
10108 10312
10109 10313
10110 10314
10111 10315
10112 10316
10113 10317
10114 10318
10115 10319
10116 10320
10117 10321
10118 10322
4


PDF LEGNTH < 5
10323 10.1136/jnnp.2003.018713 15201345.0 PMC1739089
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1739089/ www.ncbi.nlm.nih.gov
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1739089/pdf

2023-09-18 16:42:43 130-177.eduroam.rwth-aachen.de PyPDF2._reader[5571] WARNING incorrect startxref pointer(1)


10285 10489
10286 10490
4


PDF LEGNTH < 5
10491 10.1073/pnas.81.15.4998 6589643.0 PMC391620
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC391620/ www.ncbi.nlm.nih.gov
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC391620/pdf/pnas00616-0389.pdf www.ncbi.nlm.nih.gov
10287 10491


10288 10492
10289 10493
10290 10494
10291 10495
10292 10496
10293 10497
10294 10498
10295 10499
10296 10500
10297 10501
10298 10502
10299 10503
10300 10504
4


PDF LEGNTH < 5
10505 10.1177/003591577306600616 4205631.0 PMC1644972
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1644972/ www.ncbi.nlm.nih.gov
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1644972/pdf/procrsmed00333-0057.pdf www.ncbi.nlm.nih.gov
10301 10505


10302 10506
10303 10507
10304 10508
10305 10509
10306 10510
10307 10511
10308 10512
10309 10513
10310 10514
10311 10515
10312 10516
2


PDF LEGNTH < 5
10517 nan 20784000.0 PMC2163577
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2163577/ www.ncbi.nlm.nih.gov
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC

2023-09-18 16:42:48 130-177.eduroam.rwth-aachen.de PyPDF2._reader[5571] WARNING XRef object at 1812 can not be read, some object may be missing


10328 10532
10329 10533
10330 10534
10331 10535
10332 10536
10333 10537
10334 10538
10335 10539
10336 10540
10337 10541
10338 10542
10339 10543
10340 10544
10341 10545
10342 10546
10343 10547
10344 10548
10345 10549
10346 10550
10347 10551
10348 10552
10349 10553
10350 10554
10351 10555
10352 10556
10353 10557
10354 10558
10355 10559
10356 10560
10357 10561
10358 10562
10359 10563
10360 10564
10361 10565
10362 10566
10363 10567
10364 10568
10365 10569
10366 10570
10367 10571
10368 10572
10369 10573
10370 10574
10371 10575
10372 10576
10373 10577
4


PDF LEGNTH < 5
10578 10.1172/jci107745 4834890.0 PMC301542
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC301542/ www.ncbi.nlm.nih.gov
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC301542/pdf/jcinvest00159-0217.pdf www.ncbi.nlm.nih.gov
10374 10578


10375 10579
10376 10580
10377 10581
2


PDF LEGNTH < 5
10582 nan nan PMC3802865
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3802865/ www.ncbi.nlm.nih.gov
https://www.ncbi.nlm.nih.gov/pmc/articles/

In [31]:
# # step 3: process the pdf and extract text to store to a text file, record the literatures whose pdfs are not available
# input_path = fpath.poten_litera_db
# df = pd.read_csv(input_path, header=None, sep=',')
# df.columns = ["INDEX", "DOI", "PMID", "PMCID", "FULL_TEXT_URL", "FULL_TEXT_SOURCE", "PDF_URL", "PDF_SOURCE", "TITLE", "ABSTRACT", "KEYWORDS"]

# poten_litera_pdf_not_available = fpath.poten_litera_pdf_not_available
# plib.clear_file(poten_litera_pdf_not_available)

# pdf_folder = fpath.pdf_folder
# text_folder = fpath.text_folder
# start_page = 0
# end_page = 5
# text_length_to_extract = 2000

# for ind in df.index:
#     index = str(int(df.at[ind, "INDEX"]))
#     pdf_file_name = str(index) + ".pdf"
#     json_file_name = str(index) + ".json"
#     pdf_path = os.path.join(pdf_folder, pdf_file_name)
#     json_path = os.path.join(pdf_folder, json_file_name)
#     text_path = os.path.join(text_folder, str(index) + ".txt")
    
#     if os.path.exists(json_path):
#         pdf2text(pdf_path, text_path, start_page, end_page, text_length_to_extract)
#     elif os.path.exists(pdf_path):
#         json2text(json_path, text_path, start_page, end_page, text_length_to_extract)
#     else:
#         selected_row = df.iloc[[ind]]
#         selected_row.to_csv(poten_litera_pdf_not_available, mode='a', header=False, index=False)
#         print("\n")
#         print(df.at[ind, "INDEX"], df.at[ind, "DOI"], df.at[ind, "PMID"], df.at[ind, "PMCID"])
#         print(df.at[ind, "FULL_TEXT_URL"], df.at[ind, "FULL_TEXT_SOURCE"])
#         print(df.at[ind, "PDF_URL"], df.at[ind, "PDF_SOURCE"])
#         print(df.at[ind, "TITLE"], df.at[ind, "ABSTRACT"], df.at[ind, "KEYWORDS"])
#         print("\n")

In [32]:
def count_keyword(text, keyword, keyword_length):
    # remove non-alphabetic characters but keep spaces and "-"
    text = ''.join(e for e in text if (e.isalpha() or e == " " or e == "-"))
    # print(text)
    text = text.strip().lower()
    # print(text)
    
    words = []
    n = keyword_length
    n_grams = ngrams(text.split(), n)
    for gram in n_grams:
        word = gram[0]
        if n > 0:
            for i in range(1, n):
                word = word + " " + gram[i]
        words.append(word)
    # print(words)
    
    word_count = 0
    for word in words:
        # print(word)
        if word == keyword:
            word_count += 1
    return word_count
# --------------------start of test code--------------------
# # text = 'This apple 6i7s very tasty？、  2but th&e banana this is not delicious at Is all.6'
# text = "Distribution of the dopamine innervation in the macaque and human thalamus Miguel Ángel García-Cabezas,aBeatriz Rico,a,b Miguel Ángel Sánchez-González,aand Carmen Cavadaa,⁎ aDepartamento de Anatomía, Histología y Neurociencia, Facultad de Medicina, Universidad Autónoma de Madrid, C/Arzobispo Morcillo s/n, 28029 Madrid, Spain bInstituto de Neurociencias de Alicante, Universidad Miguel Hernández-CSIC, 03550 Sant Joan d ’Alacant, Spain Received 19 April 2006; revised 8 June 2006; accepted 11 July 2006 Available online 30 November 2006 We recently defined the thalamic dopaminergic system in primates; it arises from numerous dopaminergic cell groups and selectively targetsnumerous thalamic nuclei. Given the central position of the thalamus in subcortical and cortical interplay, and the functional relevance of dopamine neuromodulation in the brain, detailing dopamine dis-tribution in the thalamus should supply important information. Tothis end we performed immunohistochemistry for dopamine and the dopamine transporter in the thalamus of macaque monkeys and humans to generate maps, in the stereotaxic coronal plane, of thedistribution of dopaminergic axons. The dopamine innervation of the thalamus follows the same pattern in both species and is most dense in midline limbic nuclei, the mediodorsal and lateral posteriorassociation nuclei, and in the ventral lateral and ventral anteriormotor nuclei. This distribution suggests that thalamic dopamine has a prominent role in emotion, attention, cognition and complex somatosensory and visual processing, as well as in motor control.Most thalamic dopaminergic axons are thin and varicose and targetboth the neuropil and small blood vessels, suggesting that, besides neuronal modulation, thalamic dopamine may have a direct influence on microcirculation. The maps provided here should be a usefulreference in future experimental and neuroimaging studies aiming atclarifying the role of the thalamic dopaminergic system in health and in conditions involving brain dopamine, including Parkinson ’s disease, drug addiction and schizophrenia.© 2006 Elsevier Inc. All rights reserved. Keywords: Dopamine; Thalamus; Monkey; Human; Primate; Dopamine transporter; Parkinson; Schizophrenia; AddictionIntroduction The thalamus is made up of multiple nuclei relaying information from subcortical centers or from other cortices to the cerebral cortex (Sherman and Guillery, 2005 ), as well as the striatum, the nucleus accumbens and the amygdala ( Steriade et al., 1997 ). In addition to specific subcortical and cortical afferents, the primate thalamus receives axons containing the neuromodulators acetylcholine (Heckers et al., 1992 ), histamine ( Manning et al., 1996 ), serotonin (Morrison and Foote, 1986; Lavoie and Parent, 1991 ), and the catecholamines adrenaline ( Rico and Cavada, 1998a ), noradrenaline (Morrison and Foote, 1986; Ginsberg et al., 1993 ) and dopamine (Sánchez-González et al., 2005 ). Until recently, the existence of significant dopamine innervation in the primate thalamus has been largely ignored, probably becausedopamine innervation of the rodent thalamus is very scant(Groenewegen, 1988; Papadopoulos and Parnavelas, 1990 ). However, fragmentary data scattered through the literature endorse the presence of dopamine innervation in the primate thalamus.Postmortem biochemical studies showed the presence of dopamine in the thalamus of macaques ( Brown et al., 1979; Goldman-Rakic and Brown, 1981; Pifl et al., 1990, 1991 ) and human subjects ( Oke and Adams, 1987 ). Later, receptor binding and in situ hybridization analyses detected the presence of dopamine D2-like ( Joyce et al., 1991; Kessler et al., 1993; Hall et al., 1996; Langer et al., 1999;Rieck et al., 2004 ) and D3-like receptors ( Gurevich and Joyce, 1999 ) in several human thalamic nuclei. Positron emission tomography (PET) radioligand studies have also demonstratedthe presence of the dopamine transporter (DAT) ( Wang et al., 1995; Halldin et al., 1996; Helfenbein et al., 1999; Brownell et al., 2003 ) and of D2-like receptors ( Farde et al., 1997; Langer et al., 1999; Okubo et al., 1999; Brownell et al., 2003; Rieck et al., 2004 ) in the human and macaque thalamus. In the course of PET studies focusing on schizophrenia, D2- and D3-like radioligand binding was also found in the thalamus of control subjects ( Talvik et al., 2003; Yasuno et al., 2004 ). Finally, an immunohistochemical study using anti-DAT antibodies detected the presence of dopaminergic www.elsevier.com/locate/ynimg NeuroImage 34 (2007) 965 –984 ⁎Corresponding author. Fax: +34 91 497 53 15. E-mail address: carmen.cavada@uam.es (C. Cavada). Available online on ScienceDirect (www.sciencedirect.com). 1053-8119/$ - see front matter © 2006 Elsevier Inc. All rights reserved. doi:10.1016/j.neuroimage.2006.07.032"
# keyword = 'macaque'
# count = count_keyword(text, keyword, len(keyword.split()))
# print(count)
# ---------------------end of test code---------------------

In [33]:
def count_on_topic_kws_from_liter(text, on_topic_kws):
    text = ''.join(e for e in text if (e.isalpha() or e == " " or e == "-"))
    text = text.strip().lower()
    # print(text)

    text_length = len(text.split())
    keywords_count = dict()
    keywords_fre = dict()

    # count the on-topic keywords
    for i in range(len(on_topic_kws)):
        # print(on_topic_kws[i])
        word_count = count_keyword(text, on_topic_kws[i], len(on_topic_kws[i].split()))
        # print(word_count)
        keywords_count[on_topic_kws[i]] = word_count

    return text_length, keywords_count
# --------------------start of test code--------------------
# # text = 'Vision for action: thalamic and cortical inputs to the macaque superior tract neural tracing, parietal lobule The dorsal visual stream, the cortical circuit that in the primate brain is mainly dedicated to the visual control of actions, is split into two routes, a lateral and a medial one, both involved in coding different aspects of sensorimotor control of actions. The lateral route, named "lateral grasping network", is mainly involved in the control of the distal part of prehension, namely grasping and manipulation. The medial route, named "reach-to-grasp network", is involved in the control of the full deployment of prehension act, from the direction of arm movement to the shaping of the hand according to the object to be grasped. In macaque monkeys, the reach-to-grasp network (the target of this review) includes areas of the superior parietal lobule (SPL) that hosts visual and somatosensory neurons well suited to control goal-directed limb movements toward stationary as well as moving objects. After a brief summary of the neuronal functional properties of these areas, we will analyze their cortical and thalamic inputs thanks to retrograde neuronal tracers separately injected into the SPL areas V6, V6A, PEc, and PE. These areas receive visual and somatosensory information distributed in a caudorostral, visuosomatic trend, and some of them are directly connected with the dorsal premotor cortex. This review is particularly focused on the origin and type of visual information reaching the SPL, and on the functional role this information can play in guiding limb interaction with objects in structured and dynamic environments. Area PEc; Area V6; Area V6A; Dorsal visual stream; Goal-directed arm movement; Sensorimotor integration.'
# text = "Distribution of the dopamine innervation in the macaque and human thalamus Miguel Ángel García-Cabezas,aBeatriz Rico,a,b Miguel Ángel Sánchez-González,aand Carmen Cavadaa,⁎ aDepartamento de Anatomía, Histología y Neurociencia, Facultad de Medicina, Universidad Autónoma de Madrid, C/Arzobispo Morcillo s/n, 28029 Madrid, Spain bInstituto de Neurociencias de Alicante, Universidad Miguel Hernández-CSIC, 03550 Sant Joan d ’Alacant, Spain Received 19 April 2006; revised 8 June 2006; accepted 11 July 2006 Available online 30 November 2006 We recently defined the thalamic dopaminergic system in primates; it arises from numerous dopaminergic cell groups and selectively targetsnumerous thalamic nuclei. Given the central position of the thalamus in subcortical and cortical interplay, and the functional relevance of dopamine neuromodulation in the brain, detailing dopamine dis-tribution in the thalamus should supply important information. Tothis end we performed immunohistochemistry for dopamine and the dopamine transporter in the thalamus of macaque monkeys and humans to generate maps, in the stereotaxic coronal plane, of thedistribution of dopaminergic axons. The dopamine innervation of the thalamus follows the same pattern in both species and is most dense in midline limbic nuclei, the mediodorsal and lateral posteriorassociation nuclei, and in the ventral lateral and ventral anteriormotor nuclei. This distribution suggests that thalamic dopamine has a prominent role in emotion, attention, cognition and complex somatosensory and visual processing, as well as in motor control.Most thalamic dopaminergic axons are thin and varicose and targetboth the neuropil and small blood vessels, suggesting that, besides neuronal modulation, thalamic dopamine may have a direct influence on microcirculation. The maps provided here should be a usefulreference in future experimental and neuroimaging studies aiming atclarifying the role of the thalamic dopaminergic system in health and in conditions involving brain dopamine, including Parkinson ’s disease, drug addiction and schizophrenia.© 2006 Elsevier Inc. All rights reserved. Keywords: Dopamine; Thalamus; Monkey; Human; Primate; Dopamine transporter; Parkinson; Schizophrenia; AddictionIntroduction The thalamus is made up of multiple nuclei relaying information from subcortical centers or from other cortices to the cerebral cortex (Sherman and Guillery, 2005 ), as well as the striatum, the nucleus accumbens and the amygdala ( Steriade et al., 1997 ). In addition to specific subcortical and cortical afferents, the primate thalamus receives axons containing the neuromodulators acetylcholine (Heckers et al., 1992 ), histamine ( Manning et al., 1996 ), serotonin (Morrison and Foote, 1986; Lavoie and Parent, 1991 ), and the catecholamines adrenaline ( Rico and Cavada, 1998a ), noradrenaline (Morrison and Foote, 1986; Ginsberg et al., 1993 ) and dopamine (Sánchez-González et al., 2005 ). Until recently, the existence of significant dopamine innervation in the primate thalamus has been largely ignored, probably becausedopamine innervation of the rodent thalamus is very scant(Groenewegen, 1988; Papadopoulos and Parnavelas, 1990 ). However, fragmentary data scattered through the literature endorse the presence of dopamine innervation in the primate thalamus.Postmortem biochemical studies showed the presence of dopamine in the thalamus of macaques ( Brown et al., 1979; Goldman-Rakic and Brown, 1981; Pifl et al., 1990, 1991 ) and human subjects ( Oke and Adams, 1987 ). Later, receptor binding and in situ hybridization analyses detected the presence of dopamine D2-like ( Joyce et al., 1991; Kessler et al., 1993; Hall et al., 1996; Langer et al., 1999;Rieck et al., 2004 ) and D3-like receptors ( Gurevich and Joyce, 1999 ) in several human thalamic nuclei. Positron emission tomography (PET) radioligand studies have also demonstratedthe presence of the dopamine transporter (DAT) ( Wang et al., 1995; Halldin et al., 1996; Helfenbein et al., 1999; Brownell et al., 2003 ) and of D2-like receptors ( Farde et al., 1997; Langer et al., 1999; Okubo et al., 1999; Brownell et al., 2003; Rieck et al., 2004 ) in the human and macaque thalamus. In the course of PET studies focusing on schizophrenia, D2- and D3-like radioligand binding was also found in the thalamus of control subjects ( Talvik et al., 2003; Yasuno et al., 2004 ). Finally, an immunohistochemical study using anti-DAT antibodies detected the presence of dopaminergic www.elsevier.com/locate/ynimg NeuroImage 34 (2007) 965 –984 ⁎Corresponding author. Fax: +34 91 497 53 15. E-mail address: carmen.cavada@uam.es (C. Cavada). Available online on ScienceDirect (www.sciencedirect.com). 1053-8119/$ - see front matter © 2006 Elsevier Inc. All rights reserved. doi:10.1016/j.neuroimage.2006.07.032"
# keywords_count= count_on_topic_kws_from_liter(text, params.on_topic_kws)
# print(keywords_count)
# ---------------------end of test code---------------------

In [34]:
def calcul_index(keywords_count_or_fre, on_topic_kws_weights):
    index = 0
    for key in keywords_count_or_fre.keys():
        index += keywords_count_or_fre[key] * on_topic_kws_weights[key]
    return index
# --------------------start of test code--------------------
# keywords_count_or_fre = {}
# index = calcul_related(keywords_count_or_fre, params.on_topic_kws_weights)
# print(index)
# ---------------------end of test code---------------------

In [35]:
def weight_and_rank(input_path, output_path):
    df = pd.read_csv(input_path, header=None, sep=",")
    df.columns = ["INDEX", "DOI", "PMID", "PMCID", "FULL_TEXT_URL", "FULL_TEXT_SOURCE", "PDF_URL", "PDF_SOURCE", "TITLE", "ABSTRACT", "KEYWORDS"]

    for ind in df.index:
        text = ""
        if df.at[ind, "TITLE"] == df.at[ind, "TITLE"]:
            text = text + " " + df.at[ind, "TITLE"]
        if df.at[ind, "ABSTRACT"] == df.at[ind, "ABSTRACT"]:
            text = text + " " + df.at[ind, "ABSTRACT"]
        if df.at[ind, "KEYWORDS"] == df.at[ind, "KEYWORDS"]:
            text = text + " " + df.at[ind, "KEYWORDS"]
        # print(text)

        text_length, keywords_count = count_on_topic_kws_from_liter(text, params.on_topic_kws)
        index = calcul_index(keywords_count, params.on_topic_kws_weights)
        # print(index)

        columns = ["INDEX", "DOI", "PMID", "PMCID", "FULL_TEXT_URL", "FULL_TEXT_SOURCE", "PDF_URL", "PDF_SOURCE", "TITLE", "ABSTRACT", "KEYWORDS", "RELEVANCE_INDEX", "TEXT_LENGTH"]
        columns = columns + [key for key in keywords_count.keys()]

        row = {
            "INDEX": [df.at[ind, "INDEX"]],
            "DOI": [df.at[ind, "DOI"]],
            "PMID": [df.at[ind, "PMID"]],
            "PMCID": [df.at[ind, "PMCID"]],
            "FULL_TEXT_URL": [df.at[ind, "FULL_TEXT_URL"]],
            "FULL_TEXT_SOURCE": [df.at[ind, "FULL_TEXT_SOURCE"]],
            "PDF_URL": [df.at[ind, "PDF_URL"]],
            "PDF_SOURCE": [df.at[ind, "PDF_SOURCE"]],
            "TITLE": [df.at[ind, "TITLE"]],
            "ABSTRACT": [df.at[ind, "ABSTRACT"]],
            "KEYWORDS": [df.at[ind, "KEYWORDS"]],
            "RELEVANCE_INDEX": [index],
            "TEXT_LENGTH": [text_length]
        }
        
        for key in params.on_topic_kws:
            row[key] = [keywords_count[key]]
        # print(row)

        if not plib.add_row_to_csv(output_path, row, columns):
            print("Error detected when adding a row to csv!")
        
        line_number_in_csv = ind + 1
        print("Line number:", line_number_in_csv, " INDEX:", int(df.at[ind, "INDEX"]))

    # rank
    df = df.sort_values(by=["RELEVANCE_INDEX"], ascending=False)
    df.to_csv(output_path, header=True, index=False)
    print("Weighting and ranking the potentially related literature succeded!")
    print("Enjoy reading!")
# --------------------start of test code--------------------
# input_path = fpath.poten_litera_litera_db
# output_path = fpath.poten_litera_litera_db_ranked
# weight_and_rank(input_path, output_path)
# ---------------------end of test code---------------------

In [36]:
# # step 3: fill in columns containing information of the number of times keywords appear and assign index to each literature and rank them
# input_path = fpath.poten_litera_litera_db
# test_path = fpath.poten_litera_testing_set_300_read
# output_path = fpath.poten_litera_litera_db_ranked

# # clear file
# plib.clear_file(output_path)

# weight_and_rank(input_path, output_path, on_topic_kws_weights)